# Archived Review Scraping Experiments

#### Purpose
This notebook contains **archived and experimental versions** of the review scraping logic.

It documents the development process and preserves alternative approaches
that were tested during the project.

---

#### What This Notebook Contains
- Earlier implementations of the review scraping logic
- Alternative Selenium configurations
- Different parsing and extraction strategies
- Experimental performance and reliability tests

---

#### Role in the Project
This notebook is **not part of the active data pipeline**.
It serves as:
- Technical documentation
- Reference for future improvements
- Backup of previously working approaches

---

#### Output
- No standardized output
- Results may vary depending on the executed sections

---

#### Notes
- This notebook is kept for transparency and reproducibility




In [ ]:
import pickle
import os
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import random
from random import uniform
import pandas as pd


Define Variables

In [ ]:
#--------------HIER STADT FESTLEGEN------------------------
stadt = "Frankfurt"
#----------------------------------------------------------

# Dateipfad und Dateiname der CSV-Datei
dateipfad_input = r"Outputs_Links"
dateiname_input = "01Links_" + stadt + ".csv"

dateipfad_input = r"\Outputs_Links"
dateiname_input = "/01Links_" + stadt + ".csv"


dateipfad_output = r"\Outputs_Bewertungen\01"
dateiname_output = "Bewertungen_" + stadt + ".csv"


#print(dateiname_input) #Check

In [ ]:
# CSV-Datei importieren und als DataFrame speichern

#df_links importieren
df_links = pd.read_csv(dateipfad_input + dateiname_input) 

#df in liste umwandeln um in for-Schleife darüber iterieren zu können
list_links=df_links["Link"].tolist()

print(list_links)



#print(dateipfad_input + dateiname_input)
#print(df_links)

Automated Review Export via Link Iteration

In [ ]:
index=0
for link in list_links:

    # Definiere den Pfad zum Chrome Webdriver
    PATH = "C:\\Users\\ElianeTuchborn\\chromedriver.exe"

    # Erstelle eine Instanz der Chrome-Optionen
    options = Options()

    # Setze einige Optionen, um den Browser menschenähnlicher zu machen
    options.add_argument("--disable-blink-features=AutomationControlled")  # Verhindert die Erkennung von Automatisierung
    options.add_argument("--disable-extensions")  # Deaktiviert Browsererweiterungen
    options.add_argument("--disable-infobars")  # Deaktiviert Infobalken
    options.add_argument("--disable-notifications")  # Deaktiviert Benachrichtigungen
    options.add_argument("--disable-popup-blocking")  # Deaktiviert Pop-up-Blockierung
    options.add_argument("--disable-web-security")  # Deaktiviert die Web-Sicherheitseinstellungen
    options.add_argument("--disable-dev-shm-usage")  # Reduziert die Speichernutzung
    options.add_argument("--disable-gpu")  # Deaktiviert die GPU-Nutzung
    options.add_argument("--disable-features=VizDisplayCompositor")  # Deaktiviert den VizDisplayCompositor
    options.add_argument("--window-size=1366,768")  # Legt die Fenstergröße fest

    # Erstelle eine Instanz des Chrome Webdrivers mit den Optionen
    driver = webdriver.Chrome(PATH, options=options)

    #Cookie einladen 

    # Öffne die Website
    driver.get(link)

    # Warte
    time.sleep(uniform(3, 5))


    # Scrolle bis ans Ende der Seite
    current_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(uniform(3, 4))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == current_height:
            break
        current_height = new_height

    # Warte
    time.sleep(uniform(2, 4))

    # Finde alle Elemente mit Kommentaren, Bewertungen, Benutzernamen, ID und Datum
    container = driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[1]/div/div[4]") # Definiere Container

    comments = container.find_elements_by_class_name("text_full")
    elements = container.find_elements_by_class_name("user_info__head")
    scores = container.find_elements_by_css_selector("[data-score]")  # Finde Elemente mit "data-score" Attribut
    id_elements = container.find_elements_by_css_selector("[data-id]")  # Finde Elemente mit "data-id" Attribut

    # Längen Check
    print("Len Comments:", len(comments))
    print("Len Elements:", len(elements))
    print("Len Stars:", len(scores))
    print("Len ID:", len(id_elements))

    # Extrahierte Kommentare in Datensatz
    comment_list = [comment.text for comment in comments]
    dfcomment = pd.DataFrame(comment_list, columns=["Kommentar"])

    # Extrahierte Elemente in Datensatz und splitte nach Benutzer, Datum und Plattform
    element_list = [element.text for element in elements]
    dfelement = pd.DataFrame(element_list, columns=["Element"])
    dfelement[["Name", "Datum", "Plattform"]] = dfelement["Element"].str.split(" vor | auf ", expand=True)


    # Extrahiere den Wert des "data-score" Attributs als Text
    scores_list = [score.get_attribute("data-score") for score in scores]
    dfscores = pd.DataFrame(scores_list, columns=["Score"])

    # Extrahiere den Wert des "data-id" Attributs als Text
    id_list = [id_element.get_attribute("data-id") for id_element in id_elements]
    dfid = pd.DataFrame(id_list, columns=["ID"])

    # Datensätze horizontal aneinanderhängen
    df_bewertungen = pd.concat([dfcomment, dfelement, dfscores, dfid], axis=1)

    #Zeitangeben cleanen
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("1 Monate", "04.23")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("2 Monate", "03.23")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("3 Monate", "02.23")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("4 Monate", "01.23")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("5 Monate", "12.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("6 Monate", "11.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("7 Monate", "10.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("8 Monate", "09.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("9 Monate", "08.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("10 Monate", "07.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("11 Monate", "06.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("12 Monate", "05.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("ein Jahr", "00.22")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("2 Jahre", "00.21")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("3 Jahre", "00.20")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("4 Jahre", "00.19")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("5 Jahre", "00.18")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("6 Jahre", "00.17")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("7 Jahre", "00.16")
    df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("8 Jahre", "00.15")

    # Schließe Browser
    driver.quit()
    time.sleep(3)


    #speichern
    #df_bewertungen.to_csv(dateipfad_output + str(index) + dateiname_output)
    df_bewertungen.to_csv(dateipfad_output + "Bewertungen_" + stadt + str(index) + ".csv")


    #index um 1 erhöhen
    index += 1


In [ ]:
print(dateipfad_output + "Bewertungen_" + stadt + str(index) + ".csv")

Export Title

In [ ]:

#Zufällige Wartezeiten
PATH = "C:\\Users\\ElianeTuchborn\\chromedriver.exe"

driver = webdriver.Chrome(PATH)

driver.get("https://www.thefork.de/restaurant/weissgold-r734285")
time.sleep(5)
print(driver.title)

# Zufällige Wartezeit zwischen 2 und 5 Sekunden
wait_time = random.uniform(5, 10)
time.sleep(wait_time)

#driver.close()
#driver.quit()


Introduce Keys

In [ ]:

PATH = "C:\\Users\\ElianeTuchborn\\chromedriver.exe"

driver = webdriver.Chrome(PATH)

driver.get("https://www.thefork.de/restaurant/weissgold-r734285")

wait_time = random.uniform(5, 10)

search = driver.find_elements_by_class_name
#search.send_keys("css-bo10x1 ef3wbs40") #Tippe Text in Suchfeld
#search.send_keys(Keys.RETURN) #Enter

#print(driver.page_source) #kompletter Code der Website

#warte bis Seite geladen hat

#main = driver.find_element_by_id("main")
#print(main.text)
print(search)

driver.quit()


V1 Export Reviews

In [ ]:

PATH = "C:\\Users\\ElianeTuchborn\\chromedriver.exe" #Definiere Pfad zu Chrome Webdriver

driver = webdriver.Chrome(PATH) #erstelle Instanz zu Chrome Webdriver

driver.get("https://de.restaurantguru.com/Wolffhardt-Frankfurt/reviews?bylang=1") #öffne Website
time.sleep(2)

for i in range(13):
    htmlelement= driver.find_element_by_tag_name('html') #Scrolls down to the bottom of the page
    htmlelement.send_keys(Keys.END)
    time.sleep(2)

comments = driver.find_elements_by_class_name("text_full") #finde alle Elemente, deren Klasse "text full" ist
ratings = driver.find_elements_by_xpath("//span[contains(@class, 'stars')]/span") #
usernames = driver.find_elements_by_xpath("//a[contains(@class, 'user_info__name')]")
dates = driver.find_elements_by_xpath("//span[contains(@class, 'grey') and contains(text(), 'vor')]") #alle elemente mit Datum


print(len(comments))
print(len(ratings))
print(len(usernames))
print(len(dates))


data = { #erstelle leeres Dictionary
    "Kommentar": [],
    "Rating": [],
    "Benutzername": [],
    "Monat": []
}

# Extrahiere die Kommentare
for comment in comments:
    data["Kommentar"].append(comment.text) 

# Extrahiere die Ratings
for rating in ratings:
    data["Rating"].append(rating.get_attribute("style"))

# Extrahiere die Benutzernamen
for username in usernames:
    data["Benutzername"].append(username.text)

# Extrahiere die Monate
for date in dates:
    data["Monat"].append(date.text)
  

driver.quit() #schließe Browser

df = pd.DataFrame(data) #Mache aus Dictionarry Dataframe
print(df)


V2 Export Reviews

In [ ]:

# Definiere den Pfad zum Chrome Webdriver
PATH = "C:\\Users\\ElianeTuchborn\\chromedriver.exe"

# Erstelle eine Instanz der Chrome-Optionen
options = Options()

# Setze einige Optionen, um den Browser menschenähnlicher zu machen
options.add_argument("--disable-blink-features=AutomationControlled")  # Verhindert die Erkennung von Automatisierung
options.add_argument("--disable-extensions")  # Deaktiviert Browsererweiterungen
options.add_argument("--disable-infobars")  # Deaktiviert Infobalken
options.add_argument("--disable-notifications")  # Deaktiviert Benachrichtigungen
options.add_argument("--disable-popup-blocking")  # Deaktiviert Pop-up-Blockierung
options.add_argument("--disable-web-security")  # Deaktiviert die Web-Sicherheitseinstellungen
options.add_argument("--disable-dev-shm-usage")  # Reduziert die Speichernutzung
options.add_argument("--disable-gpu")  # Deaktiviert die GPU-Nutzung
options.add_argument("--disable-features=VizDisplayCompositor")  # Deaktiviert den VizDisplayCompositor
options.add_argument("--window-size=1366,768")  # Legt die Fenstergröße fest

# Erstelle eine Instanz des Chrome Webdrivers mit den Optionen
driver = webdriver.Chrome(PATH, options=options)

# Öffne die Website
driver.get("https://de.restaurantguru.com/Wolffhardt-Frankfurt/reviews?bylang=1")

# Warte
time.sleep(uniform(3, 5))


# Scrolle bis ans Ende der Seite
current_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(uniform(2, 4))
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == current_height:
        break
    current_height = new_height

# Warte
time.sleep(uniform(2, 4))


# Finde alle Elemente mit Kommentaren, Bewertungen, Benutzernamen und Datum
container = driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[1]/div/div[4]") #definiere Container

comments = container.find_elements_by_class_name("text_full")
elements = driver.find_elements_by_class_name("user_info__head")
#ratings = container.find_elements_by_xpath(".//span[contains(@class, 'stars')]/span")
#usernames = container.find_elements_by_xpath(".//a[contains(@class, 'user_info__name')]")
#dates = container.find_elements_by_xpath(".//span[contains(@class, 'grey') and contains(text(), 'vor')]")

#schließe Browser
driver.quit() 

#Längen Check
print("Len Comments:",len(comments))
#print("Len Ratings:",len(ratings))
#print("Len usernames:",len(usernames))
#print("Len Dates:",len(dates))
print("Len Elements:",len(elements))

#Erstelle leeren Datensatz
df = pd.DataFrame(
    {"Kommentar":[], "Element":[]
     }
)
df=df["Kommentar"]
df=df["Element"]

#Integriere Kommentare in Datnesatz
for comment in comments:
    df["Kommentar"]=df["Kommentar"].append(comment.text)

for element in elements:
    df["Element"]=df["Element"].append(element.text)


# Extrahiere die Daten
for comment, rating, username, date in zip(comments, ratings, usernames, dates):
    data["Kommentar"].append(comment.text)
    data["Rating"].append(rating.get_attribute("style"))
    data["Benutzername"].append(username.text)
    data["Monat"].append(date.text)


print(df)


V3 Export Reviews

In [ ]:

# Definiere den Pfad zum Chrome Webdriver
PATH = "C:\\Users\\ElianeTuchborn\\chromedriver.exe"

# Erstelle eine Instanz der Chrome-Optionen
options = Options()

# Setze einige Optionen, um den Browser menschenähnlicher zu machen
options.add_argument("--disable-blink-features=AutomationControlled")  # Verhindert die Erkennung von Automatisierung
options.add_argument("--disable-extensions")  # Deaktiviert Browsererweiterungen
options.add_argument("--disable-infobars")  # Deaktiviert Infobalken
options.add_argument("--disable-notifications")  # Deaktiviert Benachrichtigungen
options.add_argument("--disable-popup-blocking")  # Deaktiviert Pop-up-Blockierung
options.add_argument("--disable-web-security")  # Deaktiviert die Web-Sicherheitseinstellungen
options.add_argument("--disable-dev-shm-usage")  # Reduziert die Speichernutzung
options.add_argument("--disable-gpu")  # Deaktiviert die GPU-Nutzung
options.add_argument("--disable-features=VizDisplayCompositor")  # Deaktiviert den VizDisplayCompositor
options.add_argument("--window-size=1366,768")  # Legt die Fenstergröße fest

# Erstelle eine Instanz des Chrome Webdrivers mit den Optionen
driver = webdriver.Chrome(PATH, options=options)

#Cookie einladen 

# Öffne die Website
driver.get("https://de.restaurantguru.com/Wolffhardt-Frankfurt/reviews?bylang=1")

# Warte
time.sleep(uniform(3, 5))


# Scrolle bis ans Ende der Seite
current_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(uniform(3, 4))
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == current_height:
        break
    current_height = new_height

# Warte
time.sleep(uniform(2, 4))

# Finde alle Elemente mit Kommentaren, Bewertungen, Benutzernamen, ID und Datum
container = driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[1]/div/div[4]") # Definiere Container

comments = container.find_elements_by_class_name("text_full")
elements = container.find_elements_by_class_name("user_info__head")
scores = container.find_elements_by_css_selector("[data-score]")  # Finde Elemente mit "data-score" Attribut
id_elements = container.find_elements_by_css_selector("[data-id]")  # Finde Elemente mit "data-id" Attribut

# Längen Check
print("Len Comments:", len(comments))
print("Len Elements:", len(elements))
print("Len Stars:", len(scores))
print("Len ID:", len(id_elements))

# Extrahierte Kommentare in Datensatz
comment_list = [comment.text for comment in comments]
dfcomment = pd.DataFrame(comment_list, columns=["Kommentar"])

# Extrahierte Elemente in Datensatz und splitte nach Benutzer, Datum und Plattform
element_list = [element.text for element in elements]
dfelement = pd.DataFrame(element_list, columns=["Element"])
dfelement[["Name", "Datum", "Plattform"]] = dfelement["Element"].str.split(" vor | auf ", expand=True)


# Extrahiere den Wert des "data-score" Attributs als Text
scores_list = [score.get_attribute("data-score") for score in scores]
dfscores = pd.DataFrame(scores_list, columns=["Score"])

# Extrahiere den Wert des "data-id" Attributs als Text
id_list = [id_element.get_attribute("data-id") for id_element in id_elements]
dfid = pd.DataFrame(id_list, columns=["ID"])

# Datensätze horizontal aneinanderhängen
df = pd.concat([dfcomment, dfelement, dfscores, dfid], axis=1)

#Zeitangeben cleanen
df["Datum"]=df["Datum"].replace("1 Monate", "04.23")
df["Datum"]=df["Datum"].replace("2 Monate", "03.23")
df["Datum"]=df["Datum"].replace("3 Monate", "02.23")
df["Datum"]=df["Datum"].replace("4 Monate", "01.23")
df["Datum"]=df["Datum"].replace("5 Monate", "12.22")
df["Datum"]=df["Datum"].replace("6 Monate", "11.22")
df["Datum"]=df["Datum"].replace("7 Monate", "10.22")
df["Datum"]=df["Datum"].replace("8 Monate", "09.22")
df["Datum"]=df["Datum"].replace("9 Monate", "08.22")
df["Datum"]=df["Datum"].replace("10 Monate", "07.22")
df["Datum"]=df["Datum"].replace("11 Monate", "06.22")
df["Datum"]=df["Datum"].replace("12 Monate", "05.22")
df["Datum"]=df["Datum"].replace("ein Jahr", "00.22")
df["Datum"]=df["Datum"].replace("2 Jahre", "00.21")
df["Datum"]=df["Datum"].replace("3 Jahre", "00.20")
df["Datum"]=df["Datum"].replace("4 Jahre", "00.19")
df["Datum"]=df["Datum"].replace("5 Jahre", "00.18")
df["Datum"]=df["Datum"].replace("6 Jahre", "00.17")
df["Datum"]=df["Datum"].replace("7 Jahre", "00.16")
df["Datum"]=df["Datum"].replace("8 Jahre", "00.15")

# Schließe Browser
driver.quit()


User Agent

In [ ]:

# User-Agent ändern, um Code wie den eines normalen Browsers aussehen zu lassen. Die options-Methode in Selenium ermöglicht das Festlegen des User-Agent-Headers. 
PATH = "C:\\Users\\ElianeTuchborn\\chromedriver.exe"

options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

driver = webdriver.Chrome(PATH, options=options)
print(driver.title)
#driver.close()
#driver.quit()
